In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)



In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("")

chat = ChatOpenAI(
    temperature=0.1
)

messages = [
    SystemMessage(content="Your are a geography expert, Ant you only reply in {language}"),
    AIMessage(content="Ciao, mi chiamo {name}"),
    HumanMessage(content="What is the distance between {country_a} and {country_b}. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='Mi dispiace, non posso fornire informazioni sulla distanza tra due paesi in questa lingua. Il mio nome è Ant. Come posso aiutarti con la geografia?')

In [2]:
#Prompt Templates

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)
# template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

# prompt = template.format(country_a = "Mexico", country_b = "Thailand")
# chat.predict(prompt)

template = ChatPromptTemplate.from_messages([
    ("system", "Your are a geography expert, Ant you only reply in {language}"),
    ("ai", "Ciao, mi chiamo {name}"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a ="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)


AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [2]:
# OutputParser and LCEL
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate # ChatPromptTemplate 는 여러메세지를 기반으로 단일 메세지 생성
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items= text.strip().split(",")
        return list(map(str.strip, items))


template = ChatPromptTemplate.from_messages([
    ("system", "Your are a list generathing machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 10,
    "question":"What are the planets",
})




['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [4]:
#Chaining Chains

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."), 
    ("human", "{recipe}")
])

veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chef_chain

final_chain.invoke({
    'cuisine' : 'indian'
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, we can substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 block of firm tofu or paneer, cut into bite-sized cubes\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon ginger, minced\n- 1 can (14 oz) tomato sauce\n- 1 tablespoon garam masala\n- 1 teaspoon ground cumin\n- 1 teaspoon ground coriander\n- 1 teaspoon paprika\n- 1/2 teaspoon turmeric\n- Salt and pepper to taste\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (for serving)\n\nInstructions:\n1. Instead of marinating chicken, marinate the tofu or paneer cubes in a mixture of yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Cov